In [ ]:
import torch
import numpy as np
import pandas as pd
import os
!pip install rouge
from rouge import Rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive 
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [ ]:
if torch.cuda.is_available():
    print('is Available')

is Available


In [ ]:
input_path = '/content/MyDrive/MyDrive/nlp_project/data/Results4.csv'
df = pd.read_csv(input_path)

In [ ]:
df

,Unnamed: 0,CODE,TEXT,GPT-OUTPUT,SCORE,ENTAILMENT
0,0,#!/usr/sbin/setkey -f\n#\n# SPD for gateway A ...,I found a question and answer related to keys ...,True,1,True
1,1,string signature format identifier (as sp...,Found the answer myself in : Signatures are en...,True,1,True
2,2,HTTP/1.1 200 OK\nContent-type: text/html\nSet-...,Answer 1: if the server uses SSL/HTTPS(verifie...,True,1,True
3,3,"@ setlocal enableextensions \n@ cd /d ""%~dp0""\...",You can use a Simple Batch File. Open Notepad...,True,1,True
4,4,[Client] <=========================> [Proxy] <...,Is there a way for the server to know that req...,True,1,True
...,...,...,...,...,...,...
2807,2807,0: kd> ?? (char *)@$proc->ImageFileName\nchar ...,as i commented in latest windows type index ha...,True,1,True
2808,2808,int main() {\n int a[10];\n a[4] = 4;\n}\n\n...,I don't understand why they would be actually ...,True,1,True
2809,2809,addr_set = currentProgram.getMemory()\nf = Fil...,I have posted my solution GitHub The key is th...,False,0,False
2810,2810,cd filesystem/\ncp $(which qemu-mips-static) ....,"As far as I understand, you need to execute ju...",NaN,-1,False


In [ ]:
import matplotlib.pyplot as plt
df['SCORE'].unique()

array([ 1, -1,  0])

In [ ]:
df.head()

,Unnamed: 0,CODE,TEXT,GPT-OUTPUT,SCORE,ENTAILMENT
0,0,#!/usr/sbin/setkey -f\n#\n# SPD for gateway A ...,I found a question and answer related to keys ...,True,1,True
1,1,string signature format identifier (as sp...,Found the answer myself in : Signatures are en...,True,1,True
2,2,HTTP/1.1 200 OK\nContent-type: text/html\nSet-...,Answer 1: if the server uses SSL/HTTPS(verifie...,True,1,True
3,3,"@ setlocal enableextensions \n@ cd /d ""%~dp0""\...",You can use a Simple Batch File. Open Notepad...,True,1,True
4,4,[Client] <=========================> [Proxy] <...,Is there a way for the server to know that req...,True,1,True


In [ ]:
# !pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.0 MB/s 
     |████████████████████████████████| 7.6 MB 60.8 MB/s 
     |████████████████████████████████| 182 kB 72.7 MB/s 


In [ ]:
# !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', do_lower_case = True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
# Print the original sentence.
print(' Original: ', "chirag {code_blk} chirag ")
num_added_tokens = tokenizer.add_special_tokens({'additional_special_tokens': ['{code_blk}']})
# model.resize_token_embeddings(len(tokenizer))

# Print the sentence split into tokens.   
print('Tokenized: ', tokenizer.tokenize("chirag {code_blk} chirag "))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize("chirag {code_blk} chirag ")))

 Original:  chirag {code_blk} chirag 
Tokenized:  ['▁', 'chi', 'rag', '{code_blk}', '▁', 'chi', 'rag']
Token IDs:  [3, 1436, 6151, 32100, 3, 1436, 6151]


In [ ]:
df = df.applymap(str)

In [ ]:
from transformers import T5Config, T5ForConditionalGeneration

config = T5Config(
    vocab_size = tokenizer.vocab_size,
    pad_token_id = tokenizer.pad_token_id,
    eos_token_id = tokenizer.eos_token_id,
    decoder_start_token_id = tokenizer.pad_token_id
)

model = T5ForConditionalGeneration(config)

In [ ]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

60492288

In [ ]:
max_code_len = max([len(x.split()) for x in df.CODE.values])
max_code_len

246

In [ ]:
max_text_len = max([len(x.split()) for x in df.TEXT.values])
max_text_len

387

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size = 0.2)

In [ ]:
df.SCORE.value_counts()

1     1303
0     1014
-1     495
Name: SCORE, dtype: int64

In [ ]:
gpt_train_df = train_df[train_df['SCORE'] == '1']

In [ ]:
test_df.SCORE.value_counts()

1     263
0     198
-1    102
Name: SCORE, dtype: int64

In [ ]:
train_df.SCORE.value_counts()

1     1040
0      816
-1     393
Name: SCORE, dtype: int64

In [ ]:
gpt_train_df.SCORE.value_counts()

1    1040
Name: SCORE, dtype: int64

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class SummaryDataset(Dataset):
    def __init__(
        self,
        data: pd.DataFrame = df,
        tokenizer: T5Tokenizer = tokenizer,
        code_max_token_len: int = 256,
        text_max_token_len: int = 512
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.code_max_token_len = code_max_token_len
        self.text_max_token_len = text_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        code = data_row['CODE']

        code_encoding = tokenizer(
            code,
            max_length=self.code_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        text_encoding = tokenizer(
            data_row['TEXT'],
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = text_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100

        return dict(
            input_ids=code_encoding['input_ids'].flatten(),
            attention_mask=code_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=text_encoding['attention_mask'].flatten()
        )

dataset = SummaryDataset(train_df, tokenizer)
dataloader = DataLoader(dataset, shuffle=True, batch_size=8)

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

44

In [ ]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

num_epochs = 20
num_training_steps = num_epochs * len(dataloader)
progress_bar = tqdm(range(num_training_steps))

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.train()

for epoch in range(num_epochs):
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        logits = outputs.logits
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5_project_model.pth')
    torch.cuda.empty_cache()
    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/5640 [00:00<?, ?it/s]

epoch: 1 -- loss: 6.289092540740967
epoch: 2 -- loss: 5.556608200073242
epoch: 3 -- loss: 5.927149295806885
epoch: 4 -- loss: 6.470649719238281
epoch: 5 -- loss: 6.36622428894043
epoch: 6 -- loss: 6.692849636077881
epoch: 7 -- loss: 5.767566680908203
epoch: 8 -- loss: 6.182742595672607
epoch: 9 -- loss: 5.7781195640563965
epoch: 10 -- loss: 5.914183616638184
epoch: 11 -- loss: 5.975638389587402
epoch: 12 -- loss: 5.963046550750732
epoch: 13 -- loss: 5.583629608154297
epoch: 14 -- loss: 5.616210460662842
epoch: 15 -- loss: 4.206090927124023
epoch: 16 -- loss: 5.879652976989746
epoch: 17 -- loss: 5.886045932769775
epoch: 18 -- loss: 6.091125965118408
epoch: 19 -- loss: 6.391166687011719
epoch: 20 -- loss: 6.084970951080322


In [ ]:
# print(torch.cuda.memory_summary(device=torch.device))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 5            |        cudaMalloc retries: 5         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    4833 MB |    4833 MB |    6463 MB |    1630 MB |
|       from large pool |    4736 MB |    4736 MB |    6348 MB |    1612 MB |
|       from small pool |      97 MB |      97 MB |     115 MB |      18 MB |
|---------------------------------------------------------------------------|
| Active memory         |    4833 MB |    4833 MB |    6463 MB |    1630 MB |
|       from large pool |    4736 MB |    4736 MB |    6348 MB |

In [ ]:
def summarizeCode(code, model=model):
    code_encoding = tokenizer(
        code,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=code_encoding['input_ids'].to(device),
        attention_mask=code_encoding['attention_mask'].to(device),
        max_length=256,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )    

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [ ]:
def evaluate_model(model, test_df):
  predictions, texts = [], []
  for code, text in zip(test_df['CODE'],test_df['TEXT']):
    predictions.append(summarizeCode(code, model))
    texts.append(text)
  
  rouge_scores = []
  rouge_model = Rouge()
  for y_pred, y_test in zip(predictions, texts):
    rouge_scores.append(rouge_model.get_scores(y_pred, y_test))
    

  length_preds = len(rouge_scores)
  avg_rouge = {
    'rouge-1': 
             {'r': 0,
              'p': 0,
              'f': 0},
    'rouge-2': {'r': 0, 
                'p': 0, 
                'f': 0},
    'rouge-l': {'r': 0,
                'p': 0,
                'f': 0}}
  for score in rouge_scores:
    for key in avg_rouge.keys():
      for key_ in avg_rouge[key].keys():
        avg_rouge[key][key_] += score[0][key][key_]

  for key in avg_rouge.keys():
    for key_ in avg_rouge[key].keys():
      avg_rouge[key][key_] /= length_preds

  return avg_rouge
avg_rouge_score = evaluate_model(model, test_df)

In [ ]:
non_gpt_rouge_score = avg_rouge_score

{'rouge-1': {'r': 0.03238414042211135,
  'p': 0.35001162701782496,
  'f': 0.05571839136107565},
 'rouge-2': {'r': 0.003024379585066932,
  'p': 0.06307535942865256,
  'f': 0.005520741873765723},
 'rouge-l': {'r': 0.030705746067914083,
  'p': 0.3391822198596642,
  'f': 0.0529815211205075}}

In [ ]:
dataset = SummaryDataset(gpt_train_df, tokenizer)
dataloader = DataLoader(dataset, shuffle=True, batch_size=8)

In [ ]:
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

num_epochs = 20
num_training_steps = num_epochs * len(dataloader)
progress_bar = tqdm(range(num_training_steps))

gpt_model = T5ForConditionalGeneration(config)

optimizer = AdamW(gpt_model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt_model = gpt_model.to(device)

gpt_model.train()

for epoch in range(num_epochs):
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = gpt_model(**batch)
        logits = outputs.logits
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': gpt_model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5_project_model_gpt_filtered.pth')
    torch.cuda.empty_cache()
    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/2600 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


epoch: 1 -- loss: 7.463490009307861
epoch: 2 -- loss: 6.405827045440674
epoch: 3 -- loss: 6.269509792327881
epoch: 4 -- loss: 6.247098445892334
epoch: 5 -- loss: 5.986255645751953
epoch: 6 -- loss: 5.422191619873047
epoch: 7 -- loss: 5.934276580810547
epoch: 8 -- loss: 5.692943096160889
epoch: 9 -- loss: 5.658504486083984
epoch: 10 -- loss: 5.480810642242432
epoch: 11 -- loss: 5.606571197509766
epoch: 12 -- loss: 5.729954719543457
epoch: 13 -- loss: 5.742477893829346
epoch: 14 -- loss: 5.561882495880127
epoch: 15 -- loss: 5.556192398071289
epoch: 16 -- loss: 5.1697821617126465
epoch: 17 -- loss: 5.757019519805908
epoch: 18 -- loss: 5.557394981384277
epoch: 19 -- loss: 5.567891597747803
epoch: 20 -- loss: 5.782711982727051


In [ ]:
gpt_avg_rouge_score = evaluate_model(gpt_model, test_df)

In [ ]:
print("Rouge Score of the normal model:")
print(avg_rouge_score)
print("Rouge Score of the model trained on Entailing dataset filtered by GPT3:")
print(gpt_avg_rouge_score)

Rouge Score of the normal model:
{'rouge-1': {'r': 0.03238414042211135, 'p': 0.35001162701782496, 'f': 0.05571839136107565}, 'rouge-2': {'r': 0.003024379585066932, 'p': 0.06307535942865256, 'f': 0.005520741873765723}, 'rouge-l': {'r': 0.030705746067914083, 'p': 0.3391822198596642, 'f': 0.0529815211205075}}
Rouge Score of the model trained on Entailing dataset filtered by GPT3:
{'rouge-1': {'r': 0.020342837879855094, 'p': 0.2748721623424227, 'f': 0.03643675875299478}, 'rouge-2': {'r': 0.0020505686398679763, 'p': 0.046090033412413516, 'f': 0.003793344026313522}, 'rouge-l': {'r': 0.01968296925118043, 'p': 0.2698900237410449, 'f': 0.03529987909420535}}
